# SEGFORMER MODELING & TRAINING

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [4]:
import evaluate
from transformers import AutoImageProcessor, TFSegformerForSemanticSegmentation, create_optimizer
from transformers.keras_callbacks import KerasMetricCallback

In [5]:
dataFrame = pd.read_csv('labels.csv')
dataFrame.head()

,Unnamed: 0,label_list
0,0,NaN
1,1,accessories
2,2,bag
3,3,belt
4,4,blazer


In [6]:
ID2LABEL = {}
for i, j in dataFrame.iterrows():
    if i==0:
        ID2LABEL[i] = 'nan'
    else:
        ID2LABEL[i] = j['label_list']
ID2LABEL

{0: 'nan',
 1: 'accessories',
 2: 'bag',
 3: 'belt',
 4: 'blazer',
 5: 'blouse',
 6: 'bodysuit',
 7: 'boots',
 8: 'bra',
 9: 'bracelet',
 10: 'cape',
 11: 'cardigan',
 12: 'clogs',
 13: 'coat',
 14: 'dress',
 15: 'earrings',
 16: 'flats',
 17: 'glasses',
 18: 'gloves',
 19: 'hair',
 20: 'hat',
 21: 'heels',
 22: 'hoodie',
 23: 'intimate',
 24: 'jacket',
 25: 'jeans',
 26: 'jumper',
 27: 'leggings',
 28: 'loafers',
 29: 'necklace',
 30: 'panties',
 31: 'pants',
 32: 'pumps',
 33: 'purse',
 34: 'ring',
 35: 'romper',
 36: 'sandals',
 37: 'scarf',
 38: 'shirt',
 39: 'shoes',
 40: 'shorts',
 41: 'skin',
 42: 'skirt',
 43: 'sneakers',
 44: 'socks',
 45: 'stockings',
 46: 'suit',
 47: 'sunglasses',
 48: 'sweater',
 49: 'sweatshirt',
 50: 'swimwear',
 51: 't-shirt',
 52: 'tie',
 53: 'tights',
 54: 'top',
 55: 'vest',
 56: 'wallet',
 57: 'watch',
 58: 'wedges'}

In [7]:
LABEL2ID = {v: k for k, v in ID2LABEL.items()}
LABEL2ID

{'nan': 0,
 'accessories': 1,
 'bag': 2,
 'belt': 3,
 'blazer': 4,
 'blouse': 5,
 'bodysuit': 6,
 'boots': 7,
 'bra': 8,
 'bracelet': 9,
 'cape': 10,
 'cardigan': 11,
 'clogs': 12,
 'coat': 13,
 'dress': 14,
 'earrings': 15,
 'flats': 16,
 'glasses': 17,
 'gloves': 18,
 'hair': 19,
 'hat': 20,
 'heels': 21,
 'hoodie': 22,
 'intimate': 23,
 'jacket': 24,
 'jeans': 25,
 'jumper': 26,
 'leggings': 27,
 'loafers': 28,
 'necklace': 29,
 'panties': 30,
 'pants': 31,
 'pumps': 32,
 'purse': 33,
 'ring': 34,
 'romper': 35,
 'sandals': 36,
 'scarf': 37,
 'shirt': 38,
 'shoes': 39,
 'shorts': 40,
 'skin': 41,
 'skirt': 42,
 'sneakers': 43,
 'socks': 44,
 'stockings': 45,
 'suit': 46,
 'sunglasses': 47,
 'sweater': 48,
 'sweatshirt': 49,
 'swimwear': 50,
 't-shirt': 51,
 'tie': 52,
 'tights': 53,
 'top': 54,
 'vest': 55,
 'wallet': 56,
 'watch': 57,
 'wedges': 58}

In [8]:
NUM_CLASSES = len(ID2LABEL)
NUM_CLASSES

59

In [9]:
H,W = 512,512

In [10]:
modelID = "nvidia/segformer-b5-finetuned-cityscapes-1024-1024"
model = TFSegformerForSemanticSegmentation.from_pretrained(
    modelID,
    num_labels=NUM_CLASSES,
    id2label=ID2LABEL,
    label2id=LABEL2ID,
    ignore_mismatched_sizes=True)
model.summary()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/340M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFSegformerForSemanticSegmentation.

Some weights of TFSegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b5-finetuned-cityscapes-1024-1024 and are newly initialized because the shapes did not match:
- decode_head/classifier/kernel:0: found shape (1, 1, 768, 19) in the checkpoint and (1, 1, 768, 59) in the model instantiated
- decode_head/classifier/bias:0: found shape (19,) in the checkpoint and (59,) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_segformer_for_semantic_segmentation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 segformer (TFSegformerMain  multiple                  81443008  
 Layer)                                                          
                                                                 
 decode_head (TFSegformerDe  multiple                  3197243   
 codeHead)                                                       
                                                                 
Total params: 84640251 (322.88 MB)
Trainable params: 84638715 (322.87 MB)
Non-trainable params: 1536 (6.00 KB)
_________________________________________________________________


In [11]:
trainDataset = tf.data.Dataset.load('/content/drive/MyDrive/Colab Notebooks/SavedDataset/trainDataset')
valDataset = tf.data.Dataset.load('/content/drive/MyDrive/Colab Notebooks/SavedDataset/valDataset')

In [12]:
BATCH_SIZE = 2
NUM_EPOCHS = 15
NUM_TRAIN_STEPS = len(trainDataset)*NUM_EPOCHS
LEARNING_RATE = 6e-5
WEIGHT_DECAY = 0.01

optimizer, LR_Schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_train_steps=NUM_TRAIN_STEPS,
    weight_decay_rate=WEIGHT_DECAY,
    num_warmup_steps=0,
)

In [13]:
metric = evaluate.load("mean_iou")

In [14]:
def computeMetrics(pred):
    logits, labels = pred
    logtis = tf.transpose(logits, (0, 2, 3, 1))
    logits = tf.image.resize(logtis, tf.shape(labels)[1:], method='bilinear')
    predLabels = tf.argmax(logits, axis=-1)

    metrics = metric.compute(predictions=predLabels, references=labels, num_labels=NUM_CLASSES, ignore_index=0)

    perCategoryAccuracy = metrics.pop('per_category_accuracy').tolist()
    perCatehgoryIOU = metrics.pop('per_category_iou').tolist()

    metrics.update({f'accuracy_{ID2LABEL[i]}': v for i, v in enumerate(perCategoryAccuracy)})
    metrics.update({f'iou_{ID2LABEL[i]}': v for i, v in enumerate(perCatehgoryIOU)})

    return {'val_' + k: v for k, v in metrics.items()}


In [15]:
metricsCallback = KerasMetricCallback(
    metric_fn=computeMetrics,
    eval_dataset=valDataset,
    batch_size=BATCH_SIZE,
    label_cols=['labels'],
)
callbacks = [metricsCallback]

In [16]:
model.compile(optimizer=optimizer)

In [17]:
model.fit(trainDataset, validation_data=valDataset, epochs=NUM_EPOCHS, callbacks=callbacks)

Epoch 1/15
462/462 [==============================] - ETA: 0s - loss: 1.3815

/usr/local/lib/python3.10/dist-packages/datasets/features/image.py:348: UserWarning: Downcasting array dtype int64 to int32 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


462/462 [==============================] - 310s 430ms/step - loss: 1.3815 - val_loss: 0.6198 - val_mean_iou: 0.0527 - val_mean_accuracy: 0.0926 - val_overall_accuracy: 0.4147 - val_accuracy_nan: nan - val_accuracy_accessories: 0.0000e+00 - val_accuracy_bag: 0.7105 - val_accuracy_belt: 0.0000e+00 - val_accuracy_blazer: 0.0000e+00 - val_accuracy_blouse: 0.0060 - val_accuracy_bodysuit: 0.0000e+00 - val_accuracy_boots: 0.0000e+00 - val_accuracy_bra: nan - val_accuracy_bracelet: 0.0000e+00 - val_accuracy_cape: 0.0000e+00 - val_accuracy_cardigan: 0.0000e+00 - val_accuracy_clogs: nan - val_accuracy_coat: 0.5734 - val_accuracy_dress: 0.7726 - val_accuracy_earrings: nan - val_accuracy_flats: 0.0000e+00 - val_accuracy_glasses: 0.0000e+00 - val_accuracy_gloves: 0.0000e+00 - val_accuracy_hair: 0.4804 - val_accuracy_hat: 0.0000e+00 - val_accuracy_heels: 0.0000e+00 - val_accuracy_hoodie: nan - val_accuracy_intimate: nan - val_accuracy_jacket: 0.0000e+00 - val_accuracy_jeans: 0.0180 - val_accuracy_ju

/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


462/462 [==============================] - 131s 283ms/step - loss: 0.7471 - val_loss: 0.4933 - val_mean_iou: 0.0843 - val_mean_accuracy: 0.1322 - val_overall_accuracy: 0.4868 - val_accuracy_nan: nan - val_accuracy_accessories: 0.0000e+00 - val_accuracy_bag: 0.7977 - val_accuracy_belt: 0.0000e+00 - val_accuracy_blazer: 0.0022 - val_accuracy_blouse: 0.3605 - val_accuracy_bodysuit: 0.0000e+00 - val_accuracy_boots: 0.0000e+00 - val_accuracy_bra: nan - val_accuracy_bracelet: 0.0000e+00 - val_accuracy_cape: 0.0000e+00 - val_accuracy_cardigan: 0.0000e+00 - val_accuracy_clogs: nan - val_accuracy_coat: 0.6082 - val_accuracy_dress: 0.7221 - val_accuracy_earrings: nan - val_accuracy_flats: 0.0000e+00 - val_accuracy_glasses: 0.0000e+00 - val_accuracy_gloves: 0.0000e+00 - val_accuracy_hair: 0.7363 - val_accuracy_hat: 0.0000e+00 - val_accuracy_heels: 0.0000e+00 - val_accuracy_hoodie: nan - val_accuracy_intimate: nan - val_accuracy_jacket: 0.0000e+00 - val_accuracy_jeans: 0.3153 - val_accuracy_jumper

In [19]:
model.save('Segformer-B5.keras')